<a href="https://colab.research.google.com/github/ksm999/QuantumComputing/blob/main/My_Projection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
os.path.abspath('')

'c:\\Users\\admin\\Desktop\\Stereo'

In [ ]:
path_img = 'cat2.jpg'
path_img = os.path.join(os.path.abspath(''),
                            path_img)
print(path_img)

c:\Users\admin\Desktop\Stereo\cat2.jpg


In [ ]:
import numpy as np
import pygame
from PIL import Image

In [ ]:
path_img = 'cat2.jpg'
path_proj = 'tac.jpg'

w_proj = 400
h_proj = 300

offset_hor = 0
offset_ver = 0.4

scale = 1.5

alpha = 0 * np.pi / 180  # Angle of Rotation aroun the x-axis
beta = 2 * np.pi / 180  # Rotation angle around y-axis (around 150° to get chestnut head)
gamma = -1 * np.pi / 180  # angle of rotation about z axis

In [ ]:
def get_point_on_sphere(point,r):

    [x, y, z] = point
    k = 2 * r**2 / (x**2 + y**2 + r**2)
    return np.array([k * x, k * y, (k - 1) * r], dtype=np.float32)

def axis_rotate(point, rot_mat):

    return np.dot(rot_mat, point)


def get_pix_on_img(point, r, h_img, w_img):

    [x, y, z] = point
    if z > r:
        z = r
    row = np.arccos(z / r) / np.pi
    col = np.arctan2(y, x) / 2 / np.pi + 0.5

    row = round(row * h_img) % h_img
    col = round(col * w_img) % w_img

    return (row, col)

def projection(pix_proj, r, h_img, w_img, h_proj, w_proj):

    (row, col) = pix_proj

    x = row + (offset_ver - 0.5) * h_proj
    y = col + (offset_hor - 0.5) * w_proj
    z = 0

    Q = np.array([x, y, z], dtype=np.float32)
    P = get_point_on_sphere(Q, r)
    P = axis_rotate(P, rot_mat)
    return get_pix_on_img(P, r, h_img, w_img)


In [ ]:
arr_img = np.array(Image.open(path_img))
arr_proj = np.zeros((h_proj, w_proj, 3), dtype=np.uint8)
h_img = arr_img.shape[0]
w_img = arr_img.shape[1]

print(h_img, w_img)

720 1280


In [ ]:
arr_img = np.array(Image.open(path_img))
arr_proj = np.zeros((h_proj, w_proj, 3), dtype=np.uint8)

h_img = arr_img.shape[0]
w_img = arr_img.shape[1]

# print(h_img, w_img)  # height, width

r = min(h_proj, w_proj) / 10 * scale

# min(300, 400) / 10 * 1.5 = 300 / 10 * 1.5 = 45

rot_mat = np.array([[
    np.cos(gamma) * np.cos(beta),
    np.cos(gamma) * np.sin(beta) * np.sin(alpha) -
    np.sin(gamma) * np.cos(alpha),
    np.cos(gamma) * np.sin(beta) * np.cos(alpha) +
    np.sin(gamma) * np.sin(alpha)
    ],
                [
                        np.sin(gamma) * np.cos(beta),
                        np.sin(gamma) * np.sin(beta) * np.sin(alpha) +
                        np.cos(gamma) * np.cos(alpha),
                        np.sin(gamma) * np.sin(beta) * np.cos(alpha) -
                        np.cos(gamma) * np.sin(alpha)
                        ],

                        [
                            -np.sin(beta),
                            np.cos(beta) * np.sin(alpha),
                            np.cos(beta) * np.cos(alpha)
                        ]])


for pix_proj in np.ndindex(arr_proj.shape[:2]):
    pix_img = projection(pix_proj, r, h_img, w_img, h_proj, w_proj)
    arr_proj[pix_proj] = arr_img[pix_img]

Image.fromarray(arr_proj).show()
